In [3]:
from stl import mesh
import math
import numpy as np
from scipy import special

#Sort according to Distance/Radius (Min To Max)
def qsort(distance,x_grid,y_grid,z_grid):
    startpoint = 0
    endpoint = len(distance)
    output = np.concatenate((distance,x_grid,y_grid,z_grid),axis=1).T
    if(startpoint < endpoint):
        flag = startpoint
        for j in range((startpoint+1),(endpoint)):
            if(output[0,startpoint]>output[0,j]):
                flag = flag+1
                temp = output[:,flag].copy()
                output[:,flag] = output[:,j]
                output[:,j] = temp
        temp = output[:,startpoint].copy()
        output[:,startpoint] = output[:,flag]
        output[:,flag] = temp
        
        output_0=np.matrix(output[0,startpoint:flag]).T
        output_1=np.matrix(output[1,startpoint:flag]).T
        output_2=np.matrix(output[2,startpoint:flag]).T
        output_3=np.matrix(output[3,startpoint:flag]).T
        
        output[:,startpoint:flag] = qsort(output_0,output_1,output_2,output_3)
        
        output_flag0=np.matrix(output[0,flag+1:endpoint]).T
        output_flag1=np.matrix(output[1,flag+1:endpoint]).T
        output_flag2=np.matrix(output[2,flag+1:endpoint]).T
        output_flag3=np.matrix(output[3,flag+1:endpoint]).T

        output[:,flag+1:endpoint] = qsort(output_flag0,output_flag1,output_flag2,output_flag3)
    return(output)

#Calculate Legendre
def legendre(n,X) :
    res = []
    for m in range(n+1):
        res.append(special.lpmv(m,n,X))
    return res

#Compute spherical harmonics
def spharm(L,M,THETA,PHI):
    if ((L==0) and (M ==0) and (THETA==0) and (PHI==0)):
        L=2
        M=1
        THETA = pi/4
        PHI = pi/4
    
    Lmn=legendre(L,np.cos(PHI))
    Lmn = np.array(Lmn)
    
    if L!=0:
        Lmn=np.squeeze(Lmn[M,...])

    
    a1=((2*L+1)/(4*math.pi))
    a2=math.factorial(L-M)/math.factorial(L+M)
    C=np.sqrt(a1*a2)
    Ymn=C*Lmn*math.e**(M*THETA*1j)
    return Ymn

#Get Spherical descriptors
def spherical_descriptors(file,angle):
    your_mesh = mesh.Mesh.from_file(file)
    x1= your_mesh.v0
    y1= your_mesh.v1
    z1=your_mesh.v2

    vertices = np.stack((x1,y1,z1), axis=1)
    verti_trans = vertices.transpose((2,0,1))
    x1 = verti_trans[0].ravel()
    y1 = verti_trans[1].ravel()
    z1 = verti_trans[2].ravel()
    
    # rotated
    c, s = np.cos(angle), np.sin(angle)
    R = np.matrix([[c, -s], [s, c]])
    x1 = np.matrix(x1.T)
    y1 = np.matrix(y1.T)
    result = R* (np.array([x1,y1]))

    x1 = result[0,:]
    x1=x1.T

    y1 = result[1,:]
    y1=y1.T
    z1=np.matrix(z1).T
    
    #align coordinates
    x2 = x1-np.min(x1)
    y2 = y1-np.min(y1)
    z2 = z1-np.min(z1)
    
    #normalize to 1 and rasterize to 2Rx2Rx2R voxel grid
    max_value = np.max([np.max(x2),np.max(y2),np.max(z2)])
    R = 32
    x1 = np.round(x2/max_value*(2*R-1))
    y1 = np.round(y2/max_value*(2*R-1))
    z1 = np.round(z2/max_value*(2*R-1))

    x_grid = []
    y_grid = []
    z_grid = []
    grid = np.zeros((2*R,2*R,2*R))
    n_points = len(x1)
    for j in range(n_points):
        if(grid[int(x1[j]),int(y1[j]),int(z1[j])]==0):
            #register
            grid[int(x1[j]),int(y1[j]),int(z1[j])]=1
            x_grid.append(x1[j])
            y_grid.append(y1[j])
            z_grid.append(z1[j])

    x_grid = np.array(x_grid).ravel()
    y_grid = np.array(y_grid).ravel()
    z_grid = np.array(z_grid).ravel()

    #get center of mass
    x_center =np.mean(x_grid)
    y_center =np.mean(y_grid)
    z_center =np.mean(z_grid)

    x_grid = x_grid - x_center
    y_grid = y_grid - y_center
    z_grid = z_grid - z_center
    
    #scale and make the average distance to center of mass is R/2
    dist = np.sqrt((x_grid)**2 + (y_grid)**2 + (z_grid)**2)
    mean_dist = np.mean(dist)
    scale_ratio = (R/2)/mean_dist

    x_grid = x_grid * scale_ratio;
    y_grid = y_grid * scale_ratio;
    z_grid = z_grid * scale_ratio;

    final_dist = np.sqrt((x_grid)**2 + (y_grid)**2 + (z_grid)**2)
    final_mean_dist = np.mean(final_dist)

    distance=np.matrix(final_dist).T
    x_grid=np.matrix(x_grid).T
    y_grid=np.matrix(y_grid).T
    z_grid=np.matrix(z_grid).T

    # qsort function
    output = qsort(distance,x_grid,y_grid,z_grid)

    output1 = np.array(output).T
    dist_vector = output1[:,0]
    x     = output1[:,1]
    y     = output1[:,2]
    z     = output1[:,3]
    phi   = np.arctan2(y,x)
    aa= z/dist_vector
    theta = np.arccos(aa)
    max_l = 16
    max_r = 32
    F_r = np.zeros((max_r,1))
    sph = np.zeros((max_r,max_l))
    for idx_n in range(0,len(dist_vector)+1,100):
        idx_r = math.ceil(dist_vector[idx_n])
        for idx_l in range(0,(max_l)):
            Y_ml = 0
            for idx_m in range(-idx_l,idx_l+1):
                if(idx_m>=0):
                    Y_ml = Y_ml+spharm(idx_l,idx_m,theta[idx_n],phi[idx_n])
                else:
                    Y_temp = spharm(idx_l,-idx_m,theta[idx_n],phi[idx_n])
                    Y_ml = Y_ml+(-1)**(-idx_m) * np.conj(Y_temp)
            F_lr = Y_ml
            sph[idx_r-1,idx_l]=sph[idx_r-1,idx_l] + abs(F_lr) ** 2
    sph_des=np.sqrt(sph)
    return sph_des

desc_sph = spherical_descriptors(file,angle)